In [29]:
import numpy as np

import sys
sys.path.insert(0, '../scripts') 

from proj1_helpers import *         # Not necessary to copy function for loading CSV data anymore
from preprocessing_helpers import * # All function related to preprocessing are now in this helper script 
                                    # (in the scripts-directory)


#opther stuff to import (for testing purposes)
%matplotlib inline
import numpy as np
import matplotlib.pyplot as plt
%load_ext autoreload
%autoreload 2

from helpers import sample_data, load_data, standardize
from least_squares import least_squares
from plots import visualization
from classifiers import *


The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [30]:
def split_data(x, y, ratio, verbose = False):
    n_samples = y_train.shape[0]

    indices = np.random.permutation(n_samples)
    sub_x_1 = x[indices][:int(ratio*n_samples)]
    sub_x_2 = x[indices][int(ratio*n_samples):]

    sub_y_1 = y[indices][:int(ratio*n_samples)]
    sub_y_2 = y[indices][int(ratio*n_samples):]

    if verbose:
        print('ration:\t', ratio)
        print('ratio of samples 1st subset:\t', np.round_((sub_y_1 == 1).sum()/(ratio*n_samples), decimals=2))
        print('ratio of samples 2nd subset:\t', np.round_((sub_y_2 == 1).sum()/((1-ratio)*n_samples), decimals=2))
    return sub_x_1, sub_y_1, sub_x_2, sub_y_2

In [31]:
#logistic regression demo on fake data
clf = ClassifierLogisticRegression(
    lambda_ = 1, 
    regularizer = 'L2', 
    gamma=0.1, 
    max_iterations = 1000, 
    threshold = 1e-8
    )

## load data.
height, weight, gender = load_data()

# build sampled x and y.
seed = 1
y = np.expand_dims(gender, axis=1)
X = np.c_[height.reshape(-1), weight.reshape(-1)]
y, X = sample_data(y, X, seed, size_samples=200)
x, mean_x, std_x = standardize(X)

#train logistic regression
tx = np.c_[np.ones((y.shape[0], 1)), x]
clf.train(y_train=y, tx_train=tx)
pred = clf.predict(tx)
y = y.ravel()
print( (pred == y).sum()/ y.shape[0] )
clf.get_params_and_results(tx, tx, y, y)

Current iteration=0, loss=138.62943611198904
Current iteration=100, loss=60.247169677116474
Current iteration=200, loss=54.698477045952146
Current iteration=300, loss=62.96436436532191
Current iteration=400, loss=56.388418086224235
Current iteration=500, loss=202.12861560941403
Current iteration=600, loss=58.30823428492711
Current iteration=700, loss=60.95310612871522
Current iteration=800, loss=60.47515458056051
Current iteration=900, loss=54.89736440630447
0.905


{'accuracy_train': 0.905,
 'accuracy_test': 0.905,
 'params': {'name': 'LogisticRegression',
  'lambda_': 1,
  'regulairizer': 'L2',
  'gamma': 0.1,
  'max_iterations': 1000,
  'threshold': 1e-08}}

In [27]:
#real stuff now

TRAIN = '../data/train.csv' # due to directory structure, the data directory is now one directory above this one
TEST = '../data/test.csv'
y, x, ids_train = load_csv_data('../data/train.csv')

In [28]:
#Check what's the best way to deal wit NaNs with a simple regression

#define the possible strategies to deal with nans
strategies = [
    'NanToMean',        #replaces NaNs with the mean
    'NanToMedian',      #replaces the NaNs with the median
    'RemoveNan',        #removes the rows containing the NaNs
    'RemoveNanFeatures', #removes the columns with NaNs
    'NanTo0',           #replaces the NaNs with 0
]

regularization = [None, 'L2']
for reg in regularization:
    for strat in strategies:
        print(strat)
        clf = ClassifierLinearRegression(lambda_ = 0.2, regularizer = reg)
        y_train, tx_train = preprocessing(y, x, strat, standardize=False) 
        print('Sanity check :n = ', tx_train.shape[0], 'D = ', tx_train.shape[1])
        tx_train, y_train, tx_validation, y_validation = split_data(tx_train, y_train, ratio = 0.7, verbose = False)
        clf.train(y_train, tx_train)
        dictionary = clf.get_params_and_results(tx_train, tx_validation, y_train, y_validation)
        print('accuracy_test', np.round(dictionary['accuracy_test'], decimals = 2))
        print('regularization', clf.regularizer, '\n\n')

NanToMean
Sanity check :n =  250000 D =  30
accuracy_test 0.74
regularization None 


NanToMedian
Sanity check :n =  250000 D =  30
accuracy_test 0.74
regularization None 


RemoveNan
Sanity check :n =  68114 D =  30
accuracy_test 0.72
regularization None 


RemoveNanFeatures
Sanity check :n =  250000 D =  19
accuracy_test 0.72
regularization None 


NanTo0
Sanity check :n =  250000 D =  30
accuracy_test 0.74
regularization None 


NanToMean
Sanity check :n =  250000 D =  30
accuracy_test 0.74
regularization L2 


NanToMedian
Sanity check :n =  250000 D =  30
accuracy_test 0.74
regularization L2 


RemoveNan
Sanity check :n =  68114 D =  30
accuracy_test 0.72
regularization L2 


RemoveNanFeatures
Sanity check :n =  250000 D =  19
accuracy_test 0.73
regularization L2 


NanTo0
Sanity check :n =  250000 D =  30
accuracy_test 0.74
regularization L2 




In [32]:
#build the centroid classifier



In [33]:
clf = ClassifierLogisticRegression(
    lambda_ = 0, 
    regularizer = None, 
    gamma=0.01, 
    max_iterations = 1000, 
    threshold = 1e-8
    )

for reg in regularization:
    for strat in strategies:
        print(strat)
        #clf = ClassifierLinearRegression(lambda_ = 0.2, regularizer = reg)
        y_train, tx_train = preprocessing(y, x, strat, standardize=False) 
        print('Sanity check :n = ', tx_train.shape[0], 'D = ', tx_train.shape[1])
        tx_train, y_train, tx_validation, y_validation = split_data(tx_train, y_train, ratio = 0.7, verbose = False)
        clf.train(y_train, tx_train)
        dictionary = clf.get_params_and_results(tx_train, tx_validation, y_train, y_validation)
        print('accuracy_test', np.round(dictionary['accuracy_test'], decimals = 2))
        print('regularization', clf.regularizer, '\n\n')

NanToMean
Sanity check :n =  200 D =  2
Current iteration=0, loss=97.04060527839235
Current iteration=100, loss=41.33813105217429
Current iteration=200, loss=36.437793585483455
Current iteration=300, loss=34.84542135520159
Current iteration=400, loss=34.13521110541265
Current iteration=500, loss=33.768140259306186
Current iteration=600, loss=33.56157152166979
Current iteration=700, loss=33.43876662571297
Current iteration=800, loss=33.36293908666624
Current iteration=900, loss=33.31481791769768
accuracy_test 0.92
regularization None 


NanToMedian
Sanity check :n =  200 D =  2
Current iteration=0, loss=97.04060527839235
Current iteration=100, loss=44.14034811311781
Current iteration=200, loss=38.91498206812396
Current iteration=300, loss=37.14712427387821
Current iteration=400, loss=36.33245127858737
Current iteration=500, loss=35.898550881524656
Current iteration=600, loss=35.64720874553516
Current iteration=700, loss=35.49346666637693
Current iteration=800, loss=35.395798226993044
Cu

In [34]:
y_train, tx_train = preprocessing(y, x, 'RemoveNanFeatures', standardize=False) 
tx_train, y_train, tx_validation, y_validation = split_data(tx_train, y_train, ratio = 0.7, verbose = False)
y_train.shape, tx_train.shape

clf = ClassifierCentroids()
clf.train(tx_train, y_train)
clf.get_params_and_results(tx_train, tx_validation, y_train, y_validation)


{'accuracy_train': 0.9285714285714286,
 'accuracy_test': 0.85,
 'params': {'name': 'ClassifierCentroids'}}

In [35]:
y_train, tx_train = preprocessing(y, x, 'OnlyNanFeatures', standardize=False) 
tx_train, y_train, tx_validation, y_validation = split_data(tx_train, y_train, ratio = 0.7, verbose = False)
y_train.shape, tx_train.shape

clf = ClassifierCentroids()
clf.train(tx_train, y_train)
clf.get_params_and_results(tx_train, tx_validation, y_train, y_validation)

{'accuracy_train': 0.4928571428571429,
 'accuracy_test': 0.48333333333333334,
 'params': {'name': 'ClassifierCentroids'}}

In [36]:
#Build a bunch of classifiers, Logistic regression, L2 norm 1. 